In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import ast
from datetime import datetime
from enum import Enum
from typing import List, Optional
import gym
from gym import spaces
from stable_baselines3 import SAC
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure
from tqdm.notebook import tqdm

### Data Model Classes

In [ ]:
class InteractionType(Enum):
    """
    Enum for different types of interactions a customer can have.
    """

    NONE = "none"
    VIEW = "view"
    LIKE = "like"
    BUY = "buy"
    RATE = "rate"
    EXIT = "exit"
    SESSION_START = "session_start"
    SESSION_CLOSE = "session_close"

In [ ]:
class Interaction:
    """
    Represents a single interaction between a customer and a product.
    """

    def __init__(
        self,
        idx: str,
        timestamp: datetime,
        customer_idx: int,
        product_idx: int,
        type: InteractionType,
        value: Optional[float] = None,
        review_score: Optional[int] = None,
        city_embedding: Optional[List[float]] = None,
        state_embedding: Optional[List[float]] = None,
        zip_code_embedding: Optional[List[float]] = None,
        product_purchase_history: Optional[List[float]] = None,
        category_purchase_history: Optional[List[float]] = None,
        rate_history: Optional[List[float]] = None,
    ) -> None:
        self.idx = idx
        self.timestamp = timestamp
        self.customer_idx = customer_idx
        self.product_idx = product_idx
        self.type = type
        self.value = value
        self.review_score = review_score
        self.city_embedding = city_embedding
        self.state_embedding = state_embedding
        self.zip_code_embedding = zip_code_embedding
        self.product_purchase_history = product_purchase_history
        self.category_purchase_history = category_purchase_history
        self.rate_history = rate_history

In [ ]:
class Customer:
    def __init__(self, idx: int, zip_code: int, city: str, state: str, interactions: Optional[List[Interaction]] = None) -> None:
        self.idx = idx
        self.zip_code = zip_code
        self.city = city
        self.state = state
        self.interactions = interactions if interactions is not None else []

In [ ]:
class Category:
    def __init__(self, idx: int, name: str, desc: str) -> None:
        self.idx = idx
        self.name = name
        self.desc = desc

In [ ]:
class Product:
    def __init__(self, idx: int, name: str, desc: str, long_desc: str, category: Category, price: float) -> None:
        self.idx = idx
        self.name = name
        self.desc = desc
        self.long_desc = long_desc
        self.category = category
        self.price = price

### Data Loader Functions

In [ ]:
def load_customers(idxs: List[int] = [], include_interactions: bool = False) -> List[Customer]:
    """
    Loads customer data from a CSV file.

    Parameters:
    - idxs: List of customer indices to load.
    - include_interactions: Whether to include interactions data.

    Returns:
    - List of Customer objects.
    """
    customer_df = pd.read_csv("../data/preprocessed_data/Customer.csv")
    interaction_df = pd.read_csv("../data/preprocessed_data/Interaction.csv") if include_interactions else None

    if idxs:
        customer_df = customer_df[customer_df["idx"].isin(idxs)]

    customers = []
    for _, row in tqdm(customer_df.iterrows(), total=len(customer_df), desc="Loading Customers"):
        interactions = []
        if include_interactions:
            customer_interactions = interaction_df[interaction_df["customer_idx"] == row["idx"]]
            for _, int_row in customer_interactions.iterrows():
                interactions.append(
                    Interaction(
                        idx=int_row["idx"],
                        timestamp=datetime.strptime(int_row["timestamp"], "%Y-%m-%d %H:%M:%S"),
                        customer_idx=int_row["customer_idx"],
                        product_idx=int_row["product_idx"],
                        type=InteractionType(int_row["type"]),
                        value=int_row["value"],
                        review_score=int_row["review_score"],
                    )
                )

        customers.append(Customer(idx=row["idx"], zip_code=row["zip_code"], city=row["city"], state=row["state"], interactions=interactions))
    return customers

In [ ]:
def parse_vector_field(field: str) -> List[float]:
    """
    Parses a string representation of a list into an actual list.

    Parameters:
    - field: String representation of a list.

    Returns:
    - List of floats.
    """
    try:
        return ast.literal_eval(field) if field else []
    except (ValueError, SyntaxError):
        return []

In [ ]:
def load_interactions(idxs: List[int] = [], customer_idxs: List[int] = [], k: int = 0) -> List[Interaction]:
    """
    Loads interaction data from a CSV file.

    Parameters:
    - idxs: List of interaction indices to load.
    - customer_idxs: List of customer indices to filter interactions.
    - k: Number of interactions to load.

    Returns:
    - List of Interaction objects.
    """
    interaction_df = pd.read_csv("../data/preprocessed_data/Customer_Interactions.csv")

    if idxs:
        interaction_df = interaction_df[interaction_df["idx"].isin(idxs)]
    elif customer_idxs:
        interaction_df = interaction_df[interaction_df["customer_idx"].isin(customer_idxs)]

    if k > 0:
        interaction_df = interaction_df.sort_values(by="timestamp", ascending=False).head(k)

    interactions = []
    for _, row in tqdm(interaction_df.iterrows(), total=len(interaction_df), desc="Loading Interactions"):
        interactions.append(
            Interaction(
                idx=row["idx"],
                timestamp=datetime.strptime(row["timestamp"], "%Y-%m-%d %H:%M:%S"),
                customer_idx=row["customer_idx"],
                product_idx=row["product_idx"],
                type=InteractionType(row["type"]),
                value=row["value"],
                review_score=row["review_score"],
                city_embedding=parse_vector_field(row["city_embedding"]),
                state_embedding=parse_vector_field(row["state_embedding"]),
                zip_code_embedding=parse_vector_field(row["zip_code_embedding"]),
                product_purchase_history=parse_vector_field(row["product_purchase_history"]),
                category_purchase_history=parse_vector_field(row["category_purchase_history"]),
                rate_history=parse_vector_field(row["rate_history"]),
            )
        )
    return interactions

In [ ]:
def store_interactions(interactions: List[Interaction]):
    """
    Stores interactions data back to a CSV file.

    Parameters:
    - interactions: List of Interaction objects.
    """
    interaction_data = [
        {
            "idx": i.idx,
            "timestamp": i.timestamp.strftime("%Y-%m-%d %H:%M:%S"),
            "customer_idx": i.customer_idx,
            "product_idx": i.product_idx,
            "type": i.type.value,
            "value": i.value,
            "review_score": i.review_score,
        }
        for i in interactions
    ]
    interaction_df = pd.DataFrame(interaction_data)
    interaction_df.to_csv("../data/preprocessed_data/Interaction.csv", index=False)

In [ ]:
def load_categories(idxs: List[int] = []) -> List[Category]:
    """
    Loads category data from a CSV file.

    Parameters:
    - idxs: List of category indices to load.

    Returns:
    - List of Category objects.
    """
    category_df = pd.read_csv("../data/preprocessed_data/Category.csv")
    if idxs:
        category_df = category_df[category_df["idx"].isin(idxs)]

    categories = [Category(idx=row["idx"], name=row["name"], desc=row["desc"]) for _, row in category_df.iterrows()]
    return categories

In [ ]:
def load_products(idxs: List[int] = []) -> List[Product]:
    """
    Loads product data from a CSV file.

    Parameters:
    - idxs: List of product indices to load.

    Returns:
    - List of Product objects.
    """
    product_df = pd.read_csv("../data/preprocessed_data/Product.csv")
    category_df = pd.read_csv("../data/preprocessed_data/Category.csv")

    category_map = {row["idx"]: Category(idx=row["idx"], name=row["name"], desc=row["desc"]) for _, row in category_df.iterrows()}

    if idxs:
        product_df = product_df[product_df["idx"].isin(idxs)]

    products = []
    for _, row in product_df.iterrows():
        category = category_map.get(row["category_num_id"])
        products.append(Product(idx=row["idx"], name=row["name"], desc=row["desc"], long_desc=row["long_desc"], category=category, price=row["price"]))
    return products

In [ ]:
# Load interaction data and product data
interactions = load_interactions()
products = [p.idx for p in load_products()]

### Custom Environment

In [ ]:
class RecommendationEnv(gym.Env):
    """
    Custom Environment for a recommendation system using reinforcement learning.

    The environment simulates a user's interaction with a recommendation system,
    using the user's history and embeddings to generate observations, and
    calculates rewards based on the agent's recommended products.
    """

    def __init__(self, interactions_data: List[dict], products_data: List[int], top_k: int = 5):
        super(RecommendationEnv, self).__init__()
        self.interactions_data = interactions_data
        self.products_data = products_data
        self.top_k = top_k
        self.current_interaction_idx = 0

        # Define action space for SAC (continuous action space)
        self.action_space = spaces.Box(low=-1, high=1, shape=(top_k,), dtype=np.float32)

        # Dynamic shapes based on the length of vectorized fields in the Interaction objects
        example_interaction = interactions_data[0]
        self.city_embedding_dim = len(example_interaction.city_embedding) if example_interaction.city_embedding else 12
        self.state_embedding_dim = len(example_interaction.state_embedding) if example_interaction.state_embedding else 12
        self.zip_code_embedding_dim = len(example_interaction.zip_code_embedding) if example_interaction.zip_code_embedding else 12
        self.purchase_history_dim = len(example_interaction.product_purchase_history) if example_interaction.product_purchase_history else len(products_data)

        # Define observation space with dynamically determined shapes
        self.observation_space = spaces.Dict(
            {
                "product_purchase_history": spaces.Box(low=0, high=1, shape=(self.purchase_history_dim,), dtype=np.float32),
                "category_purchase_history": spaces.Box(low=0, high=1, shape=(self.purchase_history_dim,), dtype=np.float32),
                "rate_history": spaces.Box(low=0, high=5, shape=(self.purchase_history_dim,), dtype=np.float32),
                "city_embedding": spaces.Box(low=0, high=1, shape=(self.city_embedding_dim,), dtype=np.float32),
                "state_embedding": spaces.Box(low=0, high=1, shape=(self.state_embedding_dim,), dtype=np.float32),
                "zip_code_embedding": spaces.Box(low=0, high=1, shape=(self.zip_code_embedding_dim,), dtype=np.float32),
            }
        )

        # Reward mapping for different interaction types
        self.reward_mapping = {
            "buy": 5.0,  # High reward for purchase
            "rate": 3.0,  # Moderate reward for rating
            "like": 2.0,  # Lower reward for like
            "view": 1.0,  # Minimal reward for view
            "none": -1.0,  # Penalty for no interaction
            "exit": -2.0,  # Penalty for exit or negative feedback
        }

    def seed(self, seed=None):
        """Sets the random seed for the environment."""
        self._seed = seed
        np.random.seed(seed)

    def reset(self):
        """Resets the environment to an initial state and returns an initial observation."""
        self.current_interaction_idx = np.random.randint(len(self.interactions_data))
        return self._get_observation()

    def step(self, action):
        """
        Executes one time step within the environment.

        Parameters:
        - action: The action taken by the agent.

        Returns:
        - obs: The next observation.
        - reward: The reward received.
        - done: Whether the episode has ended.
        - info: Additional information.
        """
        # Convert continuous actions to discrete product indices
        recommended_products = np.clip((action + 1) * (len(self.products_data) / 2), 0, len(self.products_data) - 1).astype(int)
        recommended_products = recommended_products[: self.top_k]

        # Select current interaction and simulate a response
        interaction = self.interactions_data[self.current_interaction_idx]
        simulated_response = self._simulate_user_response(interaction, recommended_products)

        # Assign reward based on simulated user response
        reward = self.reward_mapping.get(simulated_response, 0.0)

        # Update index and check if the session is done
        self.current_interaction_idx = (self.current_interaction_idx + 1) % len(self.interactions_data)
        done = simulated_response == "exit"  # End episode if user exits

        obs = self._get_observation()
        return obs, reward, done, {}

    def _simulate_user_response(self, interaction, recommended_products):
        """
        Simulates a user response based on interaction history and recommended products.

        This is a simplified function where the response type is randomly chosen for demonstration.
        In a real system, it would be influenced by actual interaction patterns.
        """
        # Randomly simulate a response; replace this with a model-based response if available
        return np.random.choice(["buy", "rate", "like", "view", "none", "exit"])

    def _get_observation(self):
        """Retrieves the current observation based on the current interaction."""
        interaction = self.interactions_data[self.current_interaction_idx]
        obs = {
            "product_purchase_history": np.array(interaction.product_purchase_history or np.zeros(self.purchase_history_dim), dtype=np.float32),
            "category_purchase_history": np.array(interaction.category_purchase_history or np.zeros(self.purchase_history_dim), dtype=np.float32),
            "rate_history": np.array(interaction.rate_history or np.zeros(self.purchase_history_dim), dtype=np.float32),
            "city_embedding": np.array(interaction.city_embedding or np.zeros(self.city_embedding_dim), dtype=np.float32),
            "state_embedding": np.array(interaction.state_embedding or np.zeros(self.state_embedding_dim), dtype=np.float32),
            "zip_code_embedding": np.array(interaction.zip_code_embedding or np.zeros(self.zip_code_embedding_dim), dtype=np.float32),
        }
        return obs

    def render(self, mode="human"):
        """Renders the environment (for debugging purposes)."""
        print(f"Current Interaction Index: {self.current_interaction_idx}")
        print(f"Recommended Product Indices: {self.products_data}")

    def close(self):
        """Performs any necessary cleanup."""
        pass

### Initialize environment

In [ ]:
# Initialize the environment
env = RecommendationEnv(interactions_data=interactions, products_data=products, top_k=5)

# Wrap the environment for vectorized training
vec_env = make_vec_env(lambda: env, n_envs=1)

#### Logging

In [ ]:
# Custom callback to update tqdm progress
class TQDMProgressBarCallback(BaseCallback):
    def __init__(self, total_timesteps, verbose=0):
        super(TQDMProgressBarCallback, self).__init__(verbose)
        self.total_timesteps = total_timesteps
        self.progress_bar = None

    def _on_training_start(self) -> None:
        # Initialize tqdm progress bar at the start of training
        self.progress_bar = tqdm(total=self.total_timesteps, desc="Training Progress")

    def _on_step(self) -> bool:
        # Update progress bar by the number of timesteps
        self.progress_bar.update(self.num_timesteps - self.progress_bar.n)
        return True

    def _on_training_end(self) -> None:
        # Close the progress bar at the end of training
        self.progress_bar.close()

In [ ]:
# Custom callback to log rewards for plotting
class RewardLoggingCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(RewardLoggingCallback, self).__init__(verbose)
        self.rewards = []

    def _on_step(self) -> bool:
        # Append the latest reward to rewards list
        if "reward" in self.locals:
            self.rewards.append(self.locals["reward"].item())
        return True

In [ ]:
# Set up TensorBoard logging directory
tensorboard_log_dir = "./tensorboard_logs/"
os.makedirs(tensorboard_log_dir, exist_ok=True)

# Configure TensorBoard logger
new_logger = configure(tensorboard_log_dir, ["tensorboard"])

### Initialize model

In [ ]:
# Initialize SAC model with TensorBoard logging
model = SAC("MultiInputPolicy", vec_env, verbose=1, policy_kwargs=dict(net_arch=[256, 256]), tensorboard_log=tensorboard_log_dir)
model.set_logger(new_logger)

In [ ]:
# Initialize custom callbacks
total_timesteps = 100000
reward_callback = RewardLoggingCallback()
progress_callback = TQDMProgressBarCallback(total_timesteps=total_timesteps)

In [ ]:
# Train the model with both progress and reward logging callbacks
model.learn(total_timesteps=total_timesteps, callback=[progress_callback, reward_callback])

In [ ]:
# Save the trained model
model.save("sac_recommendation_model")

### Plots

In [ ]:
# Plotting training metrics
def plot_training_metrics(rewards):
    # Plot rewards over timesteps
    plt.figure(figsize=(10, 5))
    plt.plot(rewards, label="Reward")
    plt.xlabel("Timesteps")
    plt.ylabel("Reward")
    plt.title("Reward vs. Timesteps")
    plt.legend()
    plt.show()

In [ ]:
# Plot the rewards collected from callback
plot_training_metrics(reward_callback.rewards)